In [8]:
from keras.layers import Input, Dense
from keras.models import Model

input_all = Input(shape=(233,), name="input")

x = Dense(512,activation='relu')(input_all)

x = Dense(1024,activation= 'relu')(x)

x = Dense(512)(x)

out = Dense(4)(x)

model = Model(inputs=[input_all], outputs=out)
model.compile(optimizer='rmsprop', loss='mse')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 233)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 512)               119808    
_________________________________________________________________
dense_15 (Dense)             (None, 1024)              525312    
_________________________________________________________________
dense_16 (Dense)             (None, 512)               524800    
_________________________________________________________________
dense_17 (Dense)             (None, 4)                 2052      
Total params: 1,171,972
Trainable params: 1,171,972
Non-trainable params: 0
_________________________________________________________________


In [12]:
from emulator import CustomEmulator
import numpy as np

emul = CustomEmulator(500,10)
num_hands = 100000
total_reward = 0
y = .99
e = 0.1

old_state = np.zeros((233))
targetQ = np.zeros((4))
replay_state = []
replay_Q = []
max_replay = 5000

for i in range(num_hands):
    reward_sb = 0
    reward_bb = 0
    #Starts with dealer
    is_sb_turn = True
    
    sb_has_played = False
    bb_has_played = False
  
    emul.new_hand()
    #while hand is still at play
    while(emul.is_round_finished() == False):
           
        emul.make_features()
    
        if(is_sb_turn == True):
            #save old state to update it asap with new rew
            if(sb_has_played == True):
                old_state = sb_features
                targetQ = allQ_sb
                
                ##tochange
            sb_features = np.concatenate([emul.players_cards[1], np.concatenate(
    emul.cards_feature), np.concatenate(emul.actions_feature),np.array([1])]).reshape((1,233))
            
            #run model to choose action
            allQ_sb = model.predict(sb_features)
            action_sb = np.argmax(allQ_sb)
            #print(allQ_sb)
            
            #randomly discover new line
            if(np.random.rand(1) < e):
                action_sb = np.random.randint(0,4)
            
            emul.play_action(action_sb)
                
            
            if(sb_has_played == True):
                targetQ[0,action_sb] = y*np.max(allQ_sb)
                #replay_state.append(old_state.reshape((233,)))
                #replay_Q.append(targetQ.reshape((4,)))

            sb_has_played = True
            is_sb_turn = False
        else:
            if(bb_has_played == True):
                old_state = bb_features
                targetQ = allQ_bb
                
            bb_features = np.concatenate([emul.players_cards[0], np.concatenate(
    emul.cards_feature), np.concatenate(emul.actions_feature),np.array([0])]).reshape((1,233))
            
            allQ_bb = model.predict(bb_features)
            action_bb = np.argmax(allQ_bb)
            
            if(np.random.rand(1) < e):
                action_bb = np.random.randint(0,4)
                
            emul.play_action(action_bb)        
            
            if(bb_has_played == True):
                targetQ[0,action_bb] = y*np.max(allQ_bb)
                #replay_state.append(old_state.reshape((233,)))
                #replay_Q.append(targetQ.reshape((4,)))


            bb_has_played = True
            is_sb_turn = True
     
    r  = emul.get_sb_reward()
    total_reward +=r
    
    if(i%10000 == 0 and i > 10):
         #fit on replay
        model.fit(np.stack(replay_state),np.stack(replay_Q),verbose=1)
    
    if(i%2000== 0):
        print('round',i)
        print('Won(SB):',r,'  Total won (SB):',total_reward)
        print('Replay size:',len(replay_state))
        print(allQ_sb)
        print(emul.get_action_histories_text(hole_cards = True))

    allQ_sb[0,action_sb] = r

    replay_state.append(old_state.reshape((233,)))
    replay_Q.append(targetQ.reshape((4,)))
    
    if(bb_has_played):   
        allQ_bb[0,action_bb] = -r
        replay_state.append(old_state.reshape((233,)))
        replay_Q.append(targetQ.reshape((4,)))
    

    if(len(replay_state) > max_replay):
        replay_state = replay_state[50:]
        replay_Q = replay_Q[50:]

            

round 0
Won(SB): -20   Total won (SB): -20
Replay size: 0
[[ 0.0138792   0.02201485 -0.00844394 -0.04323523]]
{'bb_player': ['DT', 'SJ'], 'sb_player': ['HJ', 'D3']}
preflop
sb_player SMALLBLIND 10
bb_player BIGBLIND 20
sb_player CALL 20
bb_player CALL 20
flop
sb_player checks
bb_player checks
turn
sb_player checks
bb_player checks
river
sb_player checks
bb_player checks

round 2000
Won(SB): 20   Total won (SB): 2990
Replay size: 3940
[[ 0.01389124  0.02180299 -0.0091692  -0.04499093]]
{'bb_player': ['C4', 'S3'], 'sb_player': ['S7', 'HJ']}
preflop
sb_player SMALLBLIND 10
bb_player BIGBLIND 20
sb_player CALL 20
bb_player CALL 20
flop
sb_player checks
bb_player checks
turn
sb_player checks
bb_player checks
river
sb_player checks
bb_player checks

round 4000
Won(SB): 500   Total won (SB): 3680
Replay size: 5000
[[ 0.01419229  0.02200063 -0.0086503  -0.0442292 ]]
{'bb_player': ['H9', 'S4'], 'sb_player': ['S2', 'CQ']}
preflop
sb_player SMALLBLIND 10
bb_player BIGBLIND 20
sb_player CALL 20
bb

KeyboardInterrupt: 